In [ ]:
import pandas as pd

# Specify the file path
file_path = '/content/gene_associations (1).tsv.gz'

# Read the gzipped TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t', compression='gzip')

# Display the DataFrame
df.head()

,geneId,geneSymbol,DSI,DPI,PLI,protein_class_name,protein_class,NofDiseases,NofPmids
0,1,A1BG,0.700,0.538,4.991700e-09,Receptor,DTO_05007575,27,20
1,2,A2M,0.529,0.769,4.522900e-11,Enzyme modulator,DTO_05007584,147,145
2,3,A2MP1,NaN,NaN,NaN,NaN,NaN,1,1
3,9,NAT1,0.536,0.846,1.929400e-14,Enzyme,DTO_05007624,133,184
4,10,NAT2,0.451,0.885,3.274400e-06,Enzyme,DTO_05007624,311,627


In [ ]:
!pip install requests
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd
project_= pd.read_csv("/content/genes_disease.csv")
df = project_.dropna(subset=['Disease'])
df.head()

,Unnamed: 0,Gene,Disease
13,13,prss1,ORPHA:676
25,25,esrrb,ORPHA:90636
50,50,csnk2b,ORPHA:178469
58,58,csnk2a1,ORPHA:528084
76,76,gabrb2,ORPHA:442835


Orpha:  https://www.orpha.net/en/disease/detail/746

In [ ]:
def get_info(num):
  # URL of the page to scrape
  url = "https://www.orpha.net/en/disease/detail/"+str(num)
  # Send a GET request to the URL
  response = requests.get(url)
  # Parse the HTML content
  soup = BeautifulSoup(response.text, 'html.parser')
  # Find the title tag and extract its text
  title_tag = soup.find('title')
  title = title_tag.text if title_tag else None
  ##########################################################
  # Find the <strong> tag with the text "Synonym(s):"
  synonyms=None
  synonym_tag = soup.find('strong', string="Synonym(s): ")
  # If the <strong> tag is found
  if synonym_tag:
        # Find the <ul> tag following the <strong> tag
        ul_tag = synonym_tag.find_next('ul')
        # Find all <li> tags within the <ul> tag
        synonym_tags = ul_tag.find_all('li')
        # Extract and return the synonyms
        synonyms = [tag.get_text(strip=True) for tag in synonym_tags]
        synonyms= ", ".join(synonyms)
  ##########################################################
  # Find the <strong> tag with the class "d-block mb-3 service-color" containing the text "Disease definition"
  Dis_def=None
  definition_strong = soup.find('strong', class_="d-block mb-3 service-color", string="Disease definition")
  # If the <strong> tag is found
  if definition_strong:
        # Get the next <p> tag which contains the disease definition
        definition_p = definition_strong.find_next('p')
        # If <p> tag exists
        if definition_p:
            # Extract and return the disease definition
            Dis_def= definition_p.get_text(strip=True)
    # If disease definition is not found
  ##########################################################
  Clinical_des=None
  # Find the <div> tag with the class "fs-5 orpha-blue" containing the text "Clinical description"
  clinical_div = soup.find('div', class_="fs-5 orpha-blue", string="Clinical description")
  # If the <div> tag is found
  if clinical_div:
        # Get the next <p> tag which contains the clinical description
        clinical_p = clinical_div.find_next('p')
        # If <p> tag exists
        if clinical_p:
            # Extract and return the clinical description
            Clinical_des= clinical_p.get_text(strip=True)
  return title, synonyms, Dis_def, Clinical_des

In [ ]:
'''
import requests
from bs4 import BeautifulSoup
def get_name(num):
  # URL of the page to scrape
  url = "https://www.orpha.net/en/disease/detail/"+str(num)
  # Send a GET request to the URL
  response = requests.get(url)
  # Parse the HTML content
  soup = BeautifulSoup(response.text, 'html.parser')
  # Find the title tag and extract its text
  title_tag = soup.find('title')
  title = title_tag.text if title_tag else "Title not found"
  # Print the extracted title
  return title
  #print("Title:", title)
def find_dis(y):
  if y[:6] =='ORPHA:':
      return get_name(y[6:])

df['disease_name']= df.apply(lambda x: find_dis(x['Disease']), axis=1)
df.head()
'''
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

def get_name(num):
    # URL of the page to scrape
    url = "https://www.orpha.net/en/disease/detail/" + str(num)
    # Send a GET request to the URL
    response = requests.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the title tag and extract its text
    title_tag = soup.find('title')
    title = title_tag.text if title_tag else "Title not found"
    # Print the extracted title
    return title

def find_dis(y):
    if y[:6] == 'ORPHA:':
        return get_info(y[6:])
    else:
      return None, None, None, None
# Assuming df is your pandas DataFrame
# Modify df in-place, so we don't need to return a new DataFrame
# We'll use a ThreadPoolExecutor to make asynchronous requests
with ThreadPoolExecutor(max_workers=10) as executor:
    # Apply the find_dis function to each row in parallel
    df['disease_name'], df['Syn'], df['disease_des/def'], df['Clinicalfeatures/Clinical_des']  =zip(*list(executor.map(find_dis, df['Disease'])))
 #list(executor.map(find_dis, df['Disease']))



# Print the modified DataFrame
df.head()


<ipython-input-129-627ce9703448>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['disease_name'], df['Syn'], df['disease_des/def'], df['Clinicalfeatures/Clinical_des']  =zip(*list(executor.map(find_dis, df['Disease'])))
<ipython-input-129-627ce9703448>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['disease_name'], df['Syn'], df['disease_des/def'], df['Clinicalfeatures/Clinical_des']  =zip(*list(executor.map(find_dis, df['Disease'])))
<ipython-input-129-627ce9703448>:51: SettingWithCopyWarning: 


,Unnamed: 0,Gene,Disease,disease_name,Syn,disease_des/def,Clinicalfeatures/Clinical_des
13,13,prss1,ORPHA:676,Orphanet: Hereditary chronic pancreatitis,None,A rare gastroenterologic disease characterized...,"Onset of HCP is typically early in life, durin..."
25,25,esrrb,ORPHA:90636,Orphanet: Rare autosomal recessive non-syndrom...,Autosomal recessive isolated neurosensory deaf...,None,None
50,50,csnk2b,ORPHA:178469,Orphanet: Autosomal dominant non-syndromic int...,None,None,None
58,58,csnk2a1,ORPHA:528084,Orphanet: Non-specific syndromic intellectual ...,Complex neurodevelopmental disorder,A rare genetic intellectual disability charact...,None
76,76,gabrb2,ORPHA:442835,Orphanet: Non-specific early-onset epileptic e...,"Non-specific EOEE, Undetermined EOEE, Undeterm...",A rare infantile epilepsy syndrome characteriz...,None


In [ ]:
df2= df.dropna(subset=['disease_name'])
df_unique_1 = df2.drop_duplicates(subset=['disease_name'])
df_unique_1

,Unnamed: 0,Gene,Disease,disease_name,Syn,disease_des/def,Clinicalfeatures/Clinical_des
13,13,prss1,ORPHA:676,Orphanet: Hereditary chronic pancreatitis,None,A rare gastroenterologic disease characterized...,"Onset of HCP is typically early in life, durin..."
25,25,esrrb,ORPHA:90636,Orphanet: Rare autosomal recessive non-syndrom...,Autosomal recessive isolated neurosensory deaf...,None,None
50,50,csnk2b,ORPHA:178469,Orphanet: Autosomal dominant non-syndromic int...,None,None,None
58,58,csnk2a1,ORPHA:528084,Orphanet: Non-specific syndromic intellectual ...,Complex neurodevelopmental disorder,A rare genetic intellectual disability charact...,None
76,76,gabrb2,ORPHA:442835,Orphanet: Non-specific early-onset epileptic e...,"Non-specific EOEE, Undetermined EOEE, Undeterm...",A rare infantile epilepsy syndrome characteriz...,None
...,...,...,...,...,...,...,...
56605,56605,cyp51a1,ORPHA:521432,Orphanet: Congenital cataract-severe neonatal ...,None,A rare genetic disease characterized by congen...,None
56616,56616,g6pd,ORPHA:466026,Orphanet: Class I glucose-6-phosphate dehydrog...,"Class I G6PD deficiency, Severe hemolytic anem...",A rare constitutional hemolytic anemia due to ...,None
56682,56682,dhfr,ORPHA:319651,Orphanet: Constitutional megaloblastic anemia ...,"DHFR deficiency, Dihydrofolate reductase defic...",None,None
56970,56970,gp6,ORPHA:98885,Orphanet: Bleeding diathesis due to glycoprote...,None,None,None


https://www.omim.org/entry/313490

In [ ]:
import pandas as pd
# Read the contents of omim_info.txt into a Pandas DataFrame
omim_info = pd.read_csv('omim_info.txt', sep='\t',  header=3)
# Display the DataFrame
display(omim_info.head())

,# Phenotype,Gene/Locus And Other Related Symbols,MIM Number,Cyto Location
0,"17,20-lyase deficiency, isolated, 202110 (3)","CYP17A1, CYP17, P450C17",609300.0,10q24.32
1,"17-alpha-hydroxylase/17,20-lyase deficiency, 2...","CYP17A1, CYP17, P450C17",609300.0,10q24.32
2,"2,4-dienoyl-CoA reductase deficiency, 616034 (3)","NADK2, C5orf33, DECRD",615787.0,5p13.2
3,"2-methylbutyrylglycinuria, 610006 (3)","ACADSB, SBCAD",600301.0,10q26.13
4,"3-M syndrome 1, 273750 (3)","CUL7, 3M1",609577.0,6p21.1


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_description_and_clinical_features(num):
    # URL of the page to scrape
    url = f"https://api.omim.org/api/entry?mimNumber={num}&apiKey=FNQbzW08SSWjNwMbMFBrhw&include=text"
    # Send a GET request to the URL
    response = requests.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the description section
    description_section = soup.find('textsectionname', string="description")
    # If description section is found, get the content
    description_content = description_section.find_next_sibling('textsectioncontent').get_text(strip=True) if description_section else None

    # Find the clinical features section
    clinical_features_section = soup.find('textsectionname', string="clinicalFeatures")
    # If clinical features section is found, get the content
    clinical_features_content = clinical_features_section.find_next_sibling('textsectioncontent').get_text(strip=True) if clinical_features_section else None

    return description_content, clinical_features_content

get_description_and_clinical_features(str(620283))


def find_dis_omim(number_to_search):
 result_df = omim_info[omim_info['# Phenotype'].str.contains(number_to_search)]
 result_df.index=range(result_df.shape[0])
 description_, _clinical_features=get_description_and_clinical_features(number_to_search)
 return result_df['# Phenotype'][0] , result_df['Gene/Locus And Other Related Symbols'][0],description_, _clinical_features


number_to_search = "202110"
find_dis_omim(number_to_search)
find_dis_omim('620283')

def find_dis_from_omim(y):
    if y[:5] == 'OMIM:':
      try:
        return find_dis_omim(str(y[5:]))
      except:
        pass
    return None, None, None, None
df['disease_name'], df['Syn'], df['disease_des/def'], df['Clinicalfeatures/Clinical_des'] = zip(*df.apply(lambda x: find_dis_from_omim(x['Disease']), axis=1))


/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
<ipython-input-133-ba5b93bfaa3b>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['disease_name'], df['Syn'], df['disease_des/def'], df['Clinicalfeatures/Clinical_des'] = zip(*df.apply(lambda x: find_dis_from_omim(x['Disease']), axis=1))


In [ ]:
df2= df.dropna(subset=['disease_name'])
df_unique_2 = df2.drop_duplicates(subset=['disease_name'])
df_unique_2

,Unnamed: 0,Gene,Disease,disease_name,Syn,disease_des/def,Clinicalfeatures/Clinical_des
197,197,kif4a,OMIM:313490,"Taurodontism, microdontia, and dens invaginatu...","KIF4A, KIF4, XLID100, TMDI","Taurodontism, microdontia, and dens invaginatu...",{1:Casamassimo et al. (1978)} described a fami...
359,359,epha10,OMIM:620283,"?Deafness, autosomal dominant 88, 620283 (3)","EPHA10, DFNA88",Autosomal dominant deafness-88 (DFNA88) is cha...,{2:Jiang et al. (2011)} reported a large 4-gen...
657,657,slc14a1,OMIM:111000,"[Blood group, Kidd], 111000 (3)","SLC14A1, JK, UTE, UT1",The Kidd blood group locus encodes a urea tran...,Although Jk-null red blood cells have reduced ...
674,674,gabrb1,OMIM:617153,"Developmental and epileptic encephalopathy 45,...","GABRB1, DEE45",Developmental and epileptic encephalopathy-45 ...,The {2:Epi4K Consortium and Epilepsy Phenome/G...
707,707,mos,OMIM:620383,"Oocyte/zygote/embryo maturation arrest 20, 620...","MOS, MSV, OZEMA20",Oocyte/zygote/embryo maturation arrest-20 (OZE...,{3:Zhang et al. (2021)} reported 3 unrelated i...
...,...,...,...,...,...,...,...
56117,56117,rora,OMIM:618060,Intellectual developmental disorder with or wi...,"RORA, IDDECA",None,{1:Guissart et al. (2018)} reported 11 unrelat...
56313,56313,kcnu1,OMIM:620196,"Spermatogenic failure 79, 620196 (3)","KCNU1, SLO3, SPGF79",Spermatogenic failure-79 (SPGF79) is character...,{2:Lv et al. (2022)} reported a 29-year-old in...
56769,56769,map3k8,OMIM:211980,"Adenocarcinoma of lung, response to tyrosine k...","EGFR, NISBD2",Lung cancer is the leading cause of cancer dea...,{23:Joishy et al. (1977)} described identical ...
56922,56922,mdh1,OMIM:618959,?Developmental and epileptic encephalopathy 88...,"MDH1, DEE88, KAR",Developmental and epileptic encephalopathy-88 ...,{1:Broeks et al. (2019)} reported 2 affected f...


In [ ]:
concatenated_df = pd.concat([df_unique_2, df_unique_1], ignore_index=True)
concatenated_df

,Unnamed: 0,Gene,Disease,disease_name,Syn,disease_des/def,Clinicalfeatures/Clinical_des
0,197,kif4a,OMIM:313490,"Taurodontism, microdontia, and dens invaginatu...","KIF4A, KIF4, XLID100, TMDI","Taurodontism, microdontia, and dens invaginatu...",{1:Casamassimo et al. (1978)} described a fami...
1,359,epha10,OMIM:620283,"?Deafness, autosomal dominant 88, 620283 (3)","EPHA10, DFNA88",Autosomal dominant deafness-88 (DFNA88) is cha...,{2:Jiang et al. (2011)} reported a large 4-gen...
2,657,slc14a1,OMIM:111000,"[Blood group, Kidd], 111000 (3)","SLC14A1, JK, UTE, UT1",The Kidd blood group locus encodes a urea tran...,Although Jk-null red blood cells have reduced ...
3,674,gabrb1,OMIM:617153,"Developmental and epileptic encephalopathy 45,...","GABRB1, DEE45",Developmental and epileptic encephalopathy-45 ...,The {2:Epi4K Consortium and Epilepsy Phenome/G...
4,707,mos,OMIM:620383,"Oocyte/zygote/embryo maturation arrest 20, 620...","MOS, MSV, OZEMA20",Oocyte/zygote/embryo maturation arrest-20 (OZE...,{3:Zhang et al. (2021)} reported 3 unrelated i...
...,...,...,...,...,...,...,...
1499,56605,cyp51a1,ORPHA:521432,Orphanet: Congenital cataract-severe neonatal ...,None,A rare genetic disease characterized by congen...,None
1500,56616,g6pd,ORPHA:466026,Orphanet: Class I glucose-6-phosphate dehydrog...,"Class I G6PD deficiency, Severe hemolytic anem...",A rare constitutional hemolytic anemia due to ...,None
1501,56682,dhfr,ORPHA:319651,Orphanet: Constitutional megaloblastic anemia ...,"DHFR deficiency, Dihydrofolate reductase defic...",None,None
1502,56970,gp6,ORPHA:98885,Orphanet: Bleeding diathesis due to glycoprote...,None,None,None


In [ ]:
!pip install transformers sentence-transformers datasets
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import datetime
import random
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
class BertForSTS(torch.nn.Module):
    def __init__(self):
        super(BertForSTS, self).__init__()
        #self.bert = models.Transformer("bert-base-uncased", max_seq_length=512)
        self.bert = models.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', max_seq_length=1000)
        #model = AutoModel.from_pretrained(model_name)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])
    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output
# Instantiate the model and move it to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model = BertForSTS()
model.to(device)
model.eval()
def predict_similarity(sentence_pair):
  test_input = tokenizer(sentence_pair, padding=True, truncation=True, return_tensors="pt").to(device) # max_length = 512,
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
  return sim

from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/Finetunned_Bert_2.pt'
model = BertForSTS()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()

print("##########################################################################################")
example_1=[ 'A cat is walking around a house.', 'A woman is peeling potato']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

print("##########################################################################################")
example_1=[  'arterial thoracic outlet syndrome',
 '''
 The objective of this project is to study the contribution of hemorrheology to arterial and venous thrombosis using the arterio-venous (A-V)
   ''']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")


print("##########################################################################################")
example_1=['antisynthetase syndrome',
 '''
  A rare idiopathic inflammatory myopathy (IIM) historically characterized by symmetric proximal muscle weakness, elevated muscle enzymes (creatine kinase), myopathic findings on electromyography, and muscle biopsy showing endomyial infiltration composed mainly of macrophages and lymphocytes. The features are non-specific, thus the disease should be distinguished from similar entities with specific clinical, immunological, histological features, notably dermatomyositis, immune-mediated necrotizing myopathy, anti-synthetase syndrome, inclusion body myositis, and myositis associated with other connective tissue disorder.
''']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

print("##########################################################################################")
##(one of its synonym is childhood cancer)
example_1=['mismatch repair cancer syndrome 1 '
,
'''
childhood cancer''']

print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

print("##########################################################################################")
example_1=[  'polymyositis',
 '''
 A rare idiopathic inflammatory myopathy (IIM) historically characterized by symmetric proximal muscle weakness, elevated muscle enzymes (creatine kinase), myopathic findings on electromyography, and muscle biopsy showing endomyial infiltration composed mainly of macrophages and lymphocytes. The features are non-specific, thus the disease should be distinguished from similar entities with specific clinical, immunological, histological features, notably dermatomyositis, immune-mediated necrotizing myopathy, anti-synthetase syndrome, inclusion body myositis, and myositis associated with other connective tissue disorder.
   '''
]
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

def is_about_term(input_text, target_term):
  sentence_pair=[input_text, target_term]
  test_input = tokenizer(sentence_pair, padding=True, truncation=True, return_tensors="pt").to(device) # max_length = 512,
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
  return  round(sim,2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

No GPU available, using the CPU instead.


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Mounted at /content/drive


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


##########################################################################################
Sentence 1: A cat is walking around a house.
Sentence 2: A woman is peeling potato
Predicted similarity score: -0.09
##########################################################################################
Sentence 1: arterial thoracic outlet syndrome
Sentence 2: 
 The objective of this project is to study the contribution of hemorrheology to arterial and venous thrombosis using the arterio-venous (A-V)
   
Predicted similarity score: 0.35
##########################################################################################
Sentence 1: antisynthetase syndrome
Sentence 2: 
  A rare idiopathic inflammatory myopathy (IIM) historically characterized by symmetric proximal muscle weakness, elevated muscle enzymes (creatine kinase), myopathic findings on electromyography, and muscle biopsy showing endomyial infiltration composed mainly of macrophages and lymphocytes. The features are non-specific

Early onset primary dystonia (DYT1 dystonia)

In [ ]:
Dis_desc_DYT1='''
'Dystonia' describes a neurologic condition characterized by involuntary, sustained muscle contractions affecting one or more sites of the body; 'torsion' refers to the twisting nature of body movements observed in dystonia. Dystonia has been classified as primary (dystonia as the sole or major symptom) or secondary (a symptom of another disorder), and by age of onset, muscle groups affected, and mode of inheritance (Muller and Kupke, 1990; Nemeth, 2002).
'''
Dis_Clincial_DYT1='''
Primary torsion dystonia (also known as 'idiopathic' torsion dystonia; ITD) usually begins in childhood or adolescence with involuntary posturing of the trunk, neck, or limbs (Marsden et al., 1976; Nemeth, 2002). Some patients have a myostatic picture, such as was described by Wechsler and Brock (1922). Johnson et al. (1962) described an extensively affected French-Canadian family. Age of onset of affected family members ranged from 6 to 42 years and severity of the disease varied considerably, with early-onset cases being severely affected. Minor manifestations interpreted as 'formes frustes' were found in some family members. In a large North American family of non-Jewish ancestry, Brin et al. (1989) found that age of onset ranged from 4 to 43 years (mean 14.4, median 10.0). Generalization occurred within a median time of 3 years and occurred earlier in cases with onset in the leg. One 6.5-year-old was unable to walk within 3 months of onset. Batshaw et al. (1985) described a patient with severe simulated torsion dystonia as the main feature of Munchausen syndrome.

Bressman et al. (1994) analyzed the haplotype of 174 Ashkenazi Jewish individuals affected with torsion dystonia. In this group, there were 90 carriers of the haplotype and 70 noncarriers. The authors found very striking differences in the phenotype between carriers and noncarriers. The age of onset in carriers was 12.5 years versus 36.5 years in the noncarriers. In 94% of carriers, symptoms began in a limb but only rarely in the neck and larynx. In contrast, the neck, larynx, or other cranial muscles were the site of onset in 79% of noncarriers. Discriminant analysis of limb onset, leg involvement, and age at onset distinguished haplotype carriers from noncarriers with 90% accuracy. In 23 of the 70 noncarriers, the disease was familial and included brachial, cervical, laryngeal, and facial dystonia.

Cheng et al. (1996) studied 49 probands with cervical or cranial dystonia with age of onset greater than 12 years and with a positive family history. They found that age of onset of clinical symptoms was earlier by an average of 21.25 years in the second generation than in the first, and suggested that an unstable trinucleotide repeat may be involved in adult-onset primary cranial or cervical dystonia.

Grundmann et al. (2003) stated that most cases of early-onset generalized dystonia are caused by a 3-bp deletion in the DYT1 gene (delE302/303; 605204.0001). They reported 6 patients with dystonia caused by the 3-bp deletion who exhibited wide phenotypic variability: 2 had classic early-onset primary generalized dystonia, 2 had multifocal dystonia (1 with cranial and cervical muscle involvement), and 2 had only writer's cramp with mild progression.

Kostic et al. (2006) reported a large Serbian family in which 7 members carried the delE302/303 DYT1 mutation (605204.0001). However, only 2 were affected by dystonia, indicating a penetrance of 29%. One of the affected individuals had late-onset mild torticollis, and the other had generalized jerky dystonia. In addition, 3 family members with dystonia did not carry the DYT1 mutation, indicating genetic heterogeneity or possibly a psychogenic origin. Kostic et al. (2006) commented on the phenotypic variability in this family.

Zirn et al. (2008) reported an 18-year-old girl with a severe form of early-onset dystonia. She had mildly delayed early motor development with stalling of further development during the second year of life. She never learned to walk independently and became wheelchair-bound at age 5 years. At age 13, she could no longer eat or drink without assistance. At age 18, she had dysphagia, severe dysarthria, facial palsy with reduced tongue mobility, dystonic movements, multiple joint contractures, increased muscle tone, hyperreflexia, and extensor plantar responses. Cognition was normal.
'''

In [ ]:
def cal_sem_def(x):
  if x!=None:
     try:
      return is_about_term(x, Dis_desc_DYT1)
     except:
      return is_about_term(x[:1000], Dis_desc_DYT1)
  else:
    return None
def cal_sem_clinical(x):
   if x!= None:
      try:
         return  is_about_term(x, Dis_Clincial_DYT1)
      except:
         return  is_about_term(x[:1000], Dis_Clincial_DYT1)
   else:
     return None

concatenated_df['sem_def']=concatenated_df.apply(lambda x:  cal_sem_def(x[ 'disease_des/def'])  ,axis=1)
concatenated_df['sem_clinical']=concatenated_df.apply( lambda x:  cal_sem_def(x['Clinicalfeatures/Clinical_des'])  ,axis=1)

In [ ]:
concatenated_df.to_csv('concatenated_df_new.csv')

In [ ]:
concatenated_df        =  pd.read_csv('/content/concatenated_df_new.csv')
concatenated_df['Syn'] =  concatenated_df.apply(lambda x: x['Syn'].split(',')	if type(x['Syn'])==str else []  , axis=1)
concatenated_df['disease_name'] =  concatenated_df.apply(lambda x: x['disease_name'].replace('?','').replace('{','').replace('}','').replace('[','').replace(']','') , axis=1)
concatenated_df['disease_name'] =  concatenated_df.apply(lambda x: ','.join(x['disease_name'].split(',')[:-1]) if len(x['disease_name'].split(','))>1 else	x['disease_name'] , axis=1)
concatenated_df['disease_name+syn']= concatenated_df.apply(lambda x: [x['disease_name']]+ x['Syn'] , axis=1)

concatenated_df

,Unnamed: 0.1,Unnamed: 0,Gene,Disease,disease_name,Syn,disease_des/def,Clinicalfeatures/Clinical_des,sem_def,sem_clinical,disease_name+syn
0,0,197,kif4a,OMIM:313490,"Taurodontism, microdontia, and dens invaginatus","[KIF4A, KIF4, XLID100, TMDI]","Taurodontism, microdontia, and dens invaginatu...",{1:Casamassimo et al. (1978)} described a fami...,0.58,0.66,"[Taurodontism, microdontia, and dens invaginat..."
1,1,359,epha10,OMIM:620283,"Deafness, autosomal dominant 88","[EPHA10, DFNA88]",Autosomal dominant deafness-88 (DFNA88) is cha...,{2:Jiang et al. (2011)} reported a large 4-gen...,0.76,0.55,"[Deafness, autosomal dominant 88, EPHA10, DFN..."
2,2,657,slc14a1,OMIM:111000,"Blood group, Kidd","[SLC14A1, JK, UTE, UT1]",The Kidd blood group locus encodes a urea tran...,Although Jk-null red blood cells have reduced ...,0.54,0.52,"[Blood group, Kidd, SLC14A1, JK, UTE, UT1]"
3,3,674,gabrb1,OMIM:617153,Developmental and epileptic encephalopathy 45,"[GABRB1, DEE45]",Developmental and epileptic encephalopathy-45 ...,The {2:Epi4K Consortium and Epilepsy Phenome/G...,0.89,0.90,[Developmental and epileptic encephalopathy 45...
4,4,707,mos,OMIM:620383,Oocyte/zygote/embryo maturation arrest 20,"[MOS, MSV, OZEMA20]",Oocyte/zygote/embryo maturation arrest-20 (OZE...,{3:Zhang et al. (2021)} reported 3 unrelated i...,0.76,0.49,"[Oocyte/zygote/embryo maturation arrest 20, MO..."
...,...,...,...,...,...,...,...,...,...,...,...
1499,1499,56605,cyp51a1,ORPHA:521432,Orphanet: Congenital cataract-severe neonatal ...,[],A rare genetic disease characterized by congen...,NaN,0.18,NaN,[Orphanet: Congenital cataract-severe neonatal...
1500,1500,56616,g6pd,ORPHA:466026,Orphanet: Class I glucose-6-phosphate dehydrog...,"[Class I G6PD deficiency, Severe hemolytic an...",A rare constitutional hemolytic anemia due to ...,NaN,0.35,NaN,[Orphanet: Class I glucose-6-phosphate dehydro...
1501,1501,56682,dhfr,ORPHA:319651,Orphanet: Constitutional megaloblastic anemia ...,"[DHFR deficiency, Dihydrofolate reductase def...",NaN,NaN,NaN,NaN,[Orphanet: Constitutional megaloblastic anemia...
1502,1502,56970,gp6,ORPHA:98885,Orphanet: Bleeding diathesis due to glycoprote...,[],NaN,NaN,NaN,NaN,[Orphanet: Bleeding diathesis due to glycoprot...


In [ ]:
Find_dis_dict={}
for i in concatenated_df.index:
    for j in concatenated_df['disease_name+syn'][i]:
      Find_dis_dict[ j.strip().lower()   ]  =   concatenated_df['Disease'][i]
Find_dis_dict

{'taurodontism, microdontia, and dens invaginatus': 'OMIM:313490',
 'kif4a': 'OMIM:313490',
 'kif4': 'OMIM:313490',
 'xlid100': 'OMIM:313490',
 'tmdi': 'OMIM:313490',
 'deafness, autosomal dominant 88': 'OMIM:620283',
 'epha10': 'OMIM:620283',
 'dfna88': 'OMIM:620283',
 'blood group, kidd': 'OMIM:111000',
 'slc14a1': 'OMIM:111000',
 'jk': 'OMIM:111000',
 'ute': 'OMIM:111000',
 'ut1': 'OMIM:111000',
 'developmental and epileptic encephalopathy 45': 'OMIM:617153',
 'gabrb1': 'OMIM:617153',
 'dee45': 'OMIM:617153',
 'oocyte/zygote/embryo maturation arrest 20': 'OMIM:620383',
 'mos': 'OMIM:620383',
 'msv': 'OMIM:620383',
 'ozema20': 'OMIM:620383',
 'intellectual developmental disorder, x-linked syndromic, bain type': 'OMIM:300986',
 'hnrnph2': 'OMIM:300986',
 'nrph2': 'OMIM:300986',
 'mrxsb': 'OMIM:300986',
 'cerebral infarction, susceptibility to': 'OMIM:601367',
 'prkch': 'OMIM:601367',
 'pkcl': 'OMIM:601367',
 'prkcl': 'OMIM:601367',
 'dyskeratosis congenita, autosomal recessive 8': 'OM

# Literature Mining

In [ ]:
import pandas as pd
Lit_mining=pd.read_csv('/content/dyt1.csv')

import ast

def extract_names_from_string(txt_str):
    # Convert the string representation of the list to an actual list
    txt = ast.literal_eval(txt_str)
    unique_names = set()
    for sublist in txt:
        for name in sublist:
          if name.lower() not in ['dyt1','tor1a','primary dystonias'  ]:
            unique_names.add(name.strip())
    return list(unique_names)


Lit_mining['related_gene']=Lit_mining.apply(lambda x:  extract_names_from_string(x['texts']),axis=1)
display(Lit_mining['related_gene'])

0      [movement anomalies, dystonias, dyskinesias, s...
1      [dystonias, DYT24, DYT23, DYT5a, GNAL, dystoni...
2      [R149Q, jaw muscles, c.1322A > G, G > A, GNAL,...
3      [DYT24, DYT23, PNKD, GNAL, dystonic tremor, dy...
4      [HVA, GAG, human, primary dystonia, dystonia, ...
                             ...                        
380                                   [torsion dystonia]
381                        [idiopathic torsion dystonia]
382                         [primary torsional dystonia]
383                                   [primary dystonia]
384                                                   []
Name: related_gene, Length: 385, dtype: object

In [ ]:
Lit_mining['title+abstract']=Lit_mining['a.title']+Lit_mining['a.abstractText']
Dis_dic={}
all_disease= sum(concatenated_df['disease_name+syn'],[])
all_disease = [disease.strip().lower() for disease in all_disease if disease != '' and len(disease.strip()) > 2]
for  i in Lit_mining.index:
  tetxt= Lit_mining['title+abstract'][i].lower()
  for j in all_disease:
     if j in tetxt:
        if j in tetxt.split():
           dis = Find_dis_dict[ j ]
           if dis in Dis_dic: Dis_dic[dis].append( (Lit_mining['pubmed_id'][i],j))
           else:              Dis_dic[dis] = [(Lit_mining['pubmed_id'][i],j)  ]
Dis_dic.keys()

dict_keys(['ORPHA:206436', 'ORPHA:550', 'ORPHA:974', 'ORPHA:256', 'ORPHA:420429', 'ORPHA:329308', 'ORPHA:98805', 'OMIM:616093', 'ORPHA:440427', 'ORPHA:647', 'ORPHA:803', 'OMIM:112100', 'ORPHA:216866', 'ORPHA:35069', 'OMIM:617951', 'ORPHA:856', 'ORPHA:53351'])

In [ ]:
for i in list(Dis_dic.keys()):
  print(concatenated_df[ concatenated_df ['Disease']==i]['disease_name'].iloc[0])

Orphanet: Infantile Krabbe disease
Orphanet: MELAS
Orphanet: Adams-Oliver syndrome
Orphanet: Early-onset generalized limb-onset dystonia
Orphanet: Glycogen storage disease due to acid maltase deficiency
Orphanet: Fatty acid hydroxylase-associated neurodegeneration
Orphanet: Primary dystonia
Blood group, ABO system
Orphanet: Severe early-onset pulmonary alveolar proteinosis due to MARS deficiency
Orphanet: Nijmegen breakage syndrome
Orphanet: Amyotrophic lateral sclerosis
Blood group, Yt system
Orphanet: Classic pantothenate kinase-associated neurodegeneration
Orphanet: Infantile neuroaxonal dystrophy
Leukodystrophy, hypomyelinating, 15
Orphanet: NON RARE IN EUROPE: Tourette syndrome
Orphanet: X-linked dystonia-parkinsonism


In [ ]:
Dis_dic

{'ORPHA:206436': [('"18299128"', 'early-onset'),
  ('"31618684"', 'early-onset'),
  ('"22166420"', 'early-onset'),
  ('"36757831"', 'early-onset'),
  ('"20223934"', 'early-onset'),
  ('"21995941"', 'early-onset'),
  ('"19685389"', 'early-onset'),
  ('"21800139"', 'early-onset'),
  ('"11528394"', 'early-onset'),
  ('"37134150"', 'early-onset'),
  ('"25379658"', 'early-onset'),
  ('"22770546"', 'early-onset'),
  ('"32243914"', 'early-onset'),
  ('"12194383"', 'early-onset'),
  ('"10814722"', 'early-onset'),
  ('"35303767"', 'early-onset'),
  ('"23196519"', 'early-onset'),
  ('"17956903"', 'early-onset'),
  ('"36329866"', 'early-onset'),
  ('"19039992"', 'early-onset'),
  ('"18827015"', 'early-onset'),
  ('"22722545"', 'early-onset'),
  ('"20482602"', 'early-onset'),
  ('"26004199"', 'early-onset'),
  ('"15930383"', 'early-onset'),
  ('"12671990"', 'early-onset'),
  ('"24938147"', 'early-onset'),
  ('"25347348"', 'early-onset'),
  ('"28432771"', 'early-onset'),
  ('"27143436"', 'early-ons

'"The monogenic primary dystonias.""Presently, 17 distinct monogenic primary dystonias referred to as dystonias 1- 4, 5a,b, 6-8, 10-13 and 15-18 (loci DYT 1-4, 5a,b, 6-8, 10-13, 15-18) have been recognized. Twelve forms are inherited as autosomal dominant, four as autosomal recessive and one as an X-linked recessive trait. Three additional autosomal dominant forms (DYT9, DYT19 and DYT20) might exist based on linkage mapping to regions apparently different from, yet in close proximity to or overlapping with the known loci DYT18, DYT10 and DYT8. Clinically, this group of movement disorders includes pure dystonias and dystonia plus syndromes. In addition, dyskinesias (paroxysmal dystonias), although phenotypically distinct from classical dystonias, are discussed within this group. In pure dystonias, dystonia is occasionally accompanied by tremor. In dystonia plus syndromes, dystonia as the prominent sign concurs with other movement abnormalities such as myoclonus and parkinsonism. In the 

In [ ]:
concatenated_df=concatenated_df[['Gene', 'Disease', 'disease_name', 'Syn',
       'disease_des/def', 'Clinicalfeatures/Clinical_des', 'sem_def',
       'sem_clinical', 'disease_name+syn', 'Liter_chcek']]

In [ ]:
dis_potential_match= list(Dis_dic.keys() )
for i in concatenated_df.index:
   if concatenated_df['Disease'][i] in dis_potential_match:
       litt=Dis_dic[concatenated_df['Disease'][i]]
       concatenated_df.at[i,'Liter_chcek']= str(litt)
       n=0
       for j in litt:
                defi=concatenated_df['disease_des/def'][i] if type(concatenated_df['disease_des/def'][i]) ==str else '' + concatenated_df[ 'Clinicalfeatures/Clinical_des'][i]  if type(concatenated_df['Clinicalfeatures/Clinical_des'][i]) ==str else ''
                abstr=Lit_mining[Lit_mining['pubmed_id']==j[0]	]['title+abstract'].iloc[0]
                num=None
                try:
                     num=is_about_term(abstr, defi)
                except:
                     num=is_about_term(abstr[:1000], defi)

                concatenated_df.at[i,'Liter_chcek_sem_'+str(n)]=num
                n+=1

<ipython-input-117-aa4176138aee>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df.at[i,'Liter_chcek_sem_'+str(n)]=num
<ipython-input-117-aa4176138aee>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df.at[i,'Liter_chcek_sem_'+str(n)]=num
<ipython-input-117-aa4176138aee>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

KeyboardInterrupt: 

In [ ]:
concatenated_df.to_csv('concatenated_df_2.csv')

In [ ]:
import nltk
nltk.download('punkt')
def get_stem(text):
    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Get the stem of each word and join them back into a sentence
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import re

nltk.download('punkt')
def get_stem(text):
    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Get the stem of each word and join them back into a sentence
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text
list_tor1a = ['dyt1', 'tor1a', 'primary dystonias', 'DYT1 dystonia', 'TOR1A dystonia',
                  'Early-onset torsion dystonia', "Oppenheim's dystonia", 'Primary torsion dystonia',
                  'Idiopathic torsion dystonia']

def calculate_count(shared_gene_phenotype_table, Lit_mining):
    # Initialize count column to zeros
    #shared_gene_phenotype_table['count'] = 0
    # Compile a regex pattern to split text into sentences
    sentence_splitter = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    # Iterate over rows in shared_gene_phenotype_table

        # Iterate over rows in Lit_mining
    for _, lit_row in Lit_mining.iterrows():
        abstract_text = lit_row['a.abstractText'].lower()
        sentences =[lit_row['a.title'].lower()]+ sentence_splitter.split(abstract_text)
        for index, row in shared_gene_phenotype_table.iterrows():
            phenotype_values = row['phenotype']
            # Check for concurrency with phenotype_values and list_tor1a
            if any(any(phenotype.lower() in sentence and any(tor1a.lower() in sentence for tor1a in list_tor1a) for phenotype in phenotype_values) for sentence in sentences):
                shared_gene_phenotype_table.at[index, 'count_phenotype'] += 1
                #break  # Move to the next row in shared_gene_phenotype_table once count is updated

    return shared_gene_phenotype_table
#Lit_mining1=Lit_mining[Lit_mining['a.abstractText']!='""']
shared_gene_phenotype_table1=shared_gene_phenotype_table# [shared_gene_phenotype_table['phenotype']!= []]
shared_gene_phenotype_table1= calculate_count(shared_gene_phenotype_table1, Lit_mining)
shared_gene_phenotype_table1

In [ ]:
import re
list_tor1a = ['dyt1', 'tor1a', 'primary dystonias', 'DYT1 dystonia', 'TOR1A dystonia',
                  'Early-onset torsion dystonia', "Oppenheim's dystonia", 'Primary torsion dystonia',
                  'Idiopathic torsion dystonia']

def calculate_count(shared_gene_phenotype_table, Lit_mining):
    # Initialize count column to zeros
    # Compile a regex pattern to split text into sentences
    sentence_splitter = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    # Iterate over rows in shared_gene_phenotype_table

        # Iterate over rows in Lit_mining
    for _, lit_row in Lit_mining.iterrows():
        abstract_text = lit_row['a.abstractText'].lower()
        sentences =[lit_row['a.title'].lower()]+ sentence_splitter.split(abstract_text)
        for index, row in shared_gene_phenotype_table.iterrows():
            phenotype_values = row['shared_gene']
            # Check for concurrency with phenotype_values and list_tor1a
            if any(any(phenotype.lower() == word and any(tor1a.lower() == word for tor1a in list_tor1a) for word in sentence.split()) for phenotype in phenotype_values for sentence in sentences):
                shared_gene_phenotype_table.at[index, 'count_gene'] += 1
                #break  # Move to the next row in shared_gene_phenotype_table once count is updated

    return shared_gene_phenotype_table
#Lit_mining1=Lit_mining[Lit_mining['a.abstractText']!='""']
shared_gene_phenotype_table1= calculate_count(shared_gene_phenotype_table1, Lit_mining)
shared_gene_phenotype_table1['count']=shared_gene_phenotype_table1['count_phenotype'] + shared_gene_phenotype_table1['count_gene']

shared_gene_phenotype_table1

In [ ]:
shared_gene_phenotype_table1['count']=shared_gene_phenotype_table1['count_phenotype'] + shared_gene_phenotype_table1['count_gene']
shared_gene_phenotype_table1

In [ ]:
shared_gene_phenotype_table1.to_csv('shared_gene_phenotype_table.csv', index=False)

# approches

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.orpha.net/en/disease/detail/676'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the element containing the disease name
disease_name_element = soup.find('h1', class_='title')

if disease_name_element:
    disease_name = disease_name_element.text.strip()
    print(f"The name of the disease is: {disease_name}")
else:
    print("Disease name not found.")

In [ ]:
import requests

def fetch_page_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print("Failed to fetch page content. Status code:", response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

def main():
    url = "https://www.orpha.net/en/disease/detail/676"
    page_content = fetch_page_content(url)
    if page_content:
        print(page_content)

if __name__ == "__main__":
    main()

In [ ]:
import requests
from bs4 import BeautifulSoup
def get_name(num):
  # URL of the page to scrape
  url = "https://www.orpha.net/en/disease/detail/"+str(num)
  # Send a GET request to the URL
  response = requests.get(url)
  # Parse the HTML content
  soup = BeautifulSoup(response.text, 'html.parser')
  # Find the title tag and extract its text
  title_tag = soup.find('title')
  title = title_tag.text if title_tag else "Title not found"
  # Print the extracted title
  return title
  #print("Title:", title)

In [ ]:
import requests

# URL of the page
url = "https://www.orpha.net/en/disease/detail/746"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Print the content of the page
    print(response.text)
else:
    # Print an error message if the request failed
    print("Failed to fetch the page. Status code:", response.status_code)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_synonyms(num):
    # URL of the page to scrape
    url = "https://www.orpha.net/en/disease/detail/" + str(num)
    # Send a GET request to the URL
    response = requests.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the <strong> tag with the text "Synonym(s):"
    synonym_tag = soup.find('strong', string="Synonym(s): ")
    # If the <strong> tag is found
    if synonym_tag:
        # Find the <ul> tag following the <strong> tag
        ul_tag = synonym_tag.find_next('ul')
        # Find all <li> tags within the <ul> tag
        synonym_tags = ul_tag.find_all('li')
        # Extract and return the synonyms
        synonyms = [tag.get_text(strip=True) for tag in synonym_tags]
        return  ", ".join(synonyms)  #,synonyms
    else:
        return ["Synonyms not found"]

# Test the function
synonyms = get_synonyms(746)
print(synonyms)

In [ ]:
def get_disease_definition(num):
    # URL of the page to scrape
    url = "https://www.orpha.net/en/disease/detail/" + str(num)
    # Send a GET request to the URL
    response = requests.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the <strong> tag with the class "d-block mb-3 service-color" containing the text "Disease definition"
    definition_strong = soup.find('strong', class_="d-block mb-3 service-color", string="Disease definition")
    # If the <strong> tag is found
    if definition_strong:
        # Get the next <p> tag which contains the disease definition
        definition_p = definition_strong.find_next('p')
        # If <p> tag exists
        if definition_p:
            # Extract and return the disease definition
            return definition_p.get_text(strip=True)
    # If disease definition is not found
    return "Disease definition not found"

# Test the function
disease_definition = get_disease_definition(746)
print("Disease definition:", disease_definition)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_clinical_description(num):
    # URL of the page to scrape
    url = "https://www.orpha.net/en/disease/detail/" + str(num)
    # Send a GET request to the URL
    response = requests.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the <div> tag with the class "fs-5 orpha-blue" containing the text "Clinical description"
    clinical_div = soup.find('div', class_="fs-5 orpha-blue", string="Clinical description")
    # If the <div> tag is found
    if clinical_div:
        # Get the next <p> tag which contains the clinical description
        clinical_p = clinical_div.find_next('p')
        # If <p> tag exists
        if clinical_p:
            # Extract and return the clinical description
            return clinical_p.get_text(strip=True)
    # If clinical description is not found
    return "Clinical description not found"

# Test the function
clinical_description = get_clinical_description(746)
print("Clinical description:", clinical_description)